# Getter Setter Decorators and SQLITE database

In [4]:
class Employee:
    
    
    
    def __init__(self,first,last):
        self.first = first
        self.last = last
        self.email = first+'.'+ last +'@email.com'  
        
        
         
    def full_name(self):
        return f"{self.first} {self.last}"

    
emp1 = Employee('John','Smith')    
print(emp1.email)
print(emp1.full_name())

John.Smith@email.com
John Smith


now lets set the first name as Jim

emp1.first = 'Tim'
print(emp1.email)
print(emp1.full_name())

see the first and last has changed but the email has not changed and if we wanted to Tim to reflect in email then we can set it as a method like fullname. but that would mean changing the code at all places where email is called

The solution is to create a method and set it as a property decorater so email will be called as attribute

In [8]:
class Employee:
    
    
    
    def __init__(self,first,last):
        self.first = first
        self.last = last
    
    @property
    def email(self):    
        return f"{self.first}.{self.last}@email.com"
        
        
         
    def full_name(self):
        return f"{self.first} {self.last}"

emp1 = Employee('John','Smith')        
emp1.first = 'Tom'    
print(emp1.email)
print(emp1.full_name())                

Tom.Smith@email.com
Tom Smith


now lets say instead of just first name we want to set the full_name like emp1.fullname = 'Tom Ghosh' 
it will run into error, so to do that we use setters

In [24]:
class Employee:

    def __init__(self, first, last):
        self.first = first
        self.last = last

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)
    
    @fullname.setter
    def fullname(self, name):
        first, last = name.split(' ')
        self.first = first
        self.last = last

emp1 = Employee('John','Smith')        
emp1.fullname = 'Tom Ghosh'  
print(emp1.first)
print(emp1.email)
print(emp1.fullname)

Tom
Tom.Ghosh@email.com
Tom Ghosh


# SQLITE

In [46]:
class Employee_for_database:

    def __init__(self, first, last,pay):
        self.first = first
        self.last = last
        self.pay = pay

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)
    
    def __repr__(self):
        return f"{self.first}, {self.last}, {self.pay}"    

In [32]:
import sqlite3

conn = sqlite3.connect("employee.db")



In [33]:
c = conn.cursor()

In [43]:
#create a employee table
c.execute("""CREATE TABLE employee(first text, last text, pay integer)""")
conn.commit()


OperationalError: table employee already exists

In [35]:
#inserting the employee details
c.execute("INSERT INTO employee VALUES ('Ayon','Ghosh',100000)")
c.execute("INSERT INTO employee VALUES ('Koyel','Ghosh',10000)")


In [40]:
#querying the database
c.execute("SELECT * FROM employee WHERE last = 'Ghosh'")
c.fetchall()

[('Ayon', 'Ghosh', 100000), ('Koyel', 'Ghosh', 10000)]

In [44]:
c.execute("SELECT * FROM employee WHERE last = 'Ghosh'")
result = c.fetchall()
for i in result:
    print(i)

('Ayon', 'Ghosh', 100000)
('Koyel', 'Ghosh', 10000)


In [47]:
#now using the employee class
emp1 = Employee_for_database("Rivan","Ghosh",1000000)
emp2 = Employee_for_database("Jon","Doe",1000)
emp3 = Employee_for_database("Jane","Doe",2000)
print(emp3.first)
print(emp3.last)
print(emp3.pay)

Jane
Doe
2000


In [51]:
#we can use string formating to insert into database from Employee_for_database but it is not a good practise because of risk of sql injection
#attacks because we can set the values as such that could break the entire database

c.execute("INSERT INTO employee VALUES ('{}','{}',{})".format(emp1.first, emp1.last, emp1.pay))


In [52]:
#instead the correct ways are:

#first way

c.execute("INSERT INTO employee VALUES (?,?,?)",(emp2.first, emp2.last, emp2.pay))

#second way

c.execute("INSERT INTO employee VALUES (:first,:last,:pay)",{'first': emp3.first, 'last': emp3.last, 'pay':emp3.pay})

In [53]:
conn.commit()

In [54]:
c.execute("SELECT * FROM employee WHERE last = 'Doe'")

In [55]:
c.fetchall()

[('Jon', 'Doe', 1000), ('Jane', 'Doe', 2000)]

In [57]:
#directly from the class
#first way mapping the previous first way ---weird thing is u have to use the whole tuple even if u have one search criterai
c.execute("SELECT * FROM employee WHERE last = ?",('Doe',))
c.fetchall()


[('Jon', 'Doe', 1000), ('Jane', 'Doe', 2000)]

In [58]:
#second way

c.execute("SELECT * FROM employee WHERE last = :last",{'last': 'Doe'})
c.fetchall()          

[('Jon', 'Doe', 1000), ('Jane', 'Doe', 2000)]

In [67]:
# we can create in memory database that is used in testing for getting a fresh slate and u dont to keep deleting a databse file over and over

conn = sqlite3.connect(':memory:')
c.execute("""CREATE TABLE employees(first text, last text, pay integer)""")
conn.commit()


In [78]:
def insert_emp(emp):
    with conn:
        c.execute("INSERT INTO employees VALUES (:first,:last,:pay)",{'first': emp.first, 'last': emp.last, 'pay':emp.pay})
    
    
def get_emp_by_name(lastname):
    with conn:
        c.execute("SELECT * FROM employees WHERE last = :last",{'last': lastname})
        return c.fetchall()  
    
    
def update_pay(emp, pay):
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last, 'pay': pay})
        
def remove_emp(emp):
    with conn:
        c.execute("""DELETE FROM employees WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last})
        

In [79]:
emp1 = Employee_for_database("Rivan","Ghosh",1000000)
emp2 = Employee_for_database("Jon","Doe",1000)
emp3 = Employee_for_database("Jane","Doe",2000)

In [80]:
insert_emp(emp1)
insert_emp(emp2)
insert_emp(emp3)

In [81]:
emps = get_emp_by_name('Doe')
print(emps)

update_pay(emp2, 95000)
remove_emp(emp1)

emps = get_emp_by_name('Doe')
print(emps)

[('Jon', 'Doe', 95000), ('Jane', 'Doe', 2000), ('Jon', 'Doe', 95000), ('Jane', 'Doe', 2000), ('Jon', 'Doe', 1000), ('Jane', 'Doe', 2000)]
[('Jon', 'Doe', 95000), ('Jane', 'Doe', 2000), ('Jon', 'Doe', 95000), ('Jane', 'Doe', 2000), ('Jon', 'Doe', 95000), ('Jane', 'Doe', 2000)]


In [82]:
conn.close()